<a href="https://colab.research.google.com/github/Mun09/-crawling/blob/main/wtf%3F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd drive/MyDrive/underground
!pwd

/content/drive/MyDrive/underground
/content/drive/MyDrive/underground


In [13]:
!pip install datasets

In [16]:
import sys
sys.path.append('..')
from common import *
from ch4 import *
from datasets import load_dataset
import re

In [17]:
class CBOW:
  def __init__(self, vocab_size, hidden_size, window_size, corpus):
    V, H = vocab_size, hidden_size

    W_in = 0.01 * np.random.randn(V,H).astype('f')
    W_out = 0.01 * np.random.randn(H,V).astype('f')

    self.in_layers = []
    for i in range(2*window_size):
      layer = Embedding(W_in)
      self.in_layers.append(layer)
    self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)

    layers = self.in_layers + [self.ns_loss]
    self.params, self.grads = [], []

    for layer in layers:
      self.params += layer.params
      self.grads += layer.grads

    self.word_vecs = W_in

  def forward(self, contexts, target):
    h = 0
    for i, layer in enumerate(self.in_layers):
      h += layer.forward(contexts[:, i])
    h *= 1/len(self.in_layers)
    loss = self.ns_loss.forward(h, target)
    return loss

  def backward(self, dout=1):
    dout = self.ns_loss.backward(dout)
    dout *= 1/len(self.in_layers)
    for layer in self.in_layers:
      layer.backward(dout)
    return None


In [18]:
window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 10

ptb = load_dataset("ptb")

corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)

contexts, target = create_contexts_target(corpus, window_size)
if config.GPU:
  contexts, target = to_gpu(contexts), to_gpu(target)

model = CBOW(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

DatasetNotFoundError: Dataset 'ptb' doesn't exist on the Hub or cannot be accessed. If the dataset is private or gated, make sure to log in with `huggingface-cli login` or visit the dataset page at https://huggingface.co/datasets/ptb to ask for access.